In [1]:
!pip install tensorflow opencv-python

In [2]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import os
import glob


2025-05-09 15:39:21.230308: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746805161.446801      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746805161.506972      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
import os
import cv2
from tqdm import tqdm

# Paths to violence and non-violence video folders
violence_dir = '/kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence'
nonviolence_dir = '/kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/NonViolence'

# Output directories for frames
output_base = '/kaggle/working/frames'
violence_output = os.path.join(output_base, 'Violence')
nonviolence_output = os.path.join(output_base, 'NonViolence')

# Create output directories
os.makedirs(violence_output, exist_ok=True)
os.makedirs(nonviolence_output, exist_ok=True)

# Function to extract N evenly spaced frames from a video
def extract_frames(video_path, output_folder, video_label, num_frames=5, resize_shape=(224, 224)):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Get frame indices to extract
    if total_frames < num_frames or total_frames == 0:
        return  # Skip videos that are too short or corrupt
    frame_indices = [int(i * total_frames / num_frames) for i in range(num_frames)]

    frame_count = 0
    saved = 0
    success = True

    while success and saved < num_frames:
        success, frame = cap.read()
        if frame_count in frame_indices:
            frame = cv2.resize(frame, resize_shape)
            video_name = os.path.basename(video_path).split('.')[0]
            frame_filename = f"{video_name}_frame_{saved}.jpg"
            frame_path = os.path.join(output_folder, frame_filename)
            cv2.imwrite(frame_path, frame)
            saved += 1
        frame_count += 1

    cap.release()

# Process all videos
print("Extracting frames from Violence videos...")
for video_file in tqdm(os.listdir(violence_dir)):
    video_path = os.path.join(violence_dir, video_file)
    extract_frames(video_path, violence_output, 'Violence')

print("Extracting frames from NonViolence videos...")
for video_file in tqdm(os.listdir(nonviolence_dir)):
    video_path = os.path.join(nonviolence_dir, video_file)
    extract_frames(video_path, nonviolence_output, 'NonViolence')


Extracting frames from Violence videos...


 43%|████▎     | 426/1000 [00:50<01:18,  7.34it/s]

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Paths
source_dir = '/kaggle/working/processed_data'
train_dir = '/kaggle/working/data/train'
val_dir = '/kaggle/working/data/val'

# Create the train and validation directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Subdirectories for each class (Violence/NonViolence)
for subdir in ['Violence', 'NonViolence']:
    os.makedirs(os.path.join(train_dir, subdir), exist_ok=True)
    os.makedirs(os.path.join(val_dir, subdir), exist_ok=True)

# Helper function to split files
def split_data(source_class_dir, train_class_dir, val_class_dir, test_size=0.2):
    # Get all the files in the source directory
    all_files = [f for f in os.listdir(source_class_dir) if os.path.isfile(os.path.join(source_class_dir, f))]
    
    # Split the files into train and validation sets
    train_files, val_files = train_test_split(all_files, test_size=test_size, random_state=42)

    # Move files to respective directories
    for file in train_files:
        shutil.copy(os.path.join(source_class_dir, file), os.path.join(train_class_dir, file))

    for file in val_files:
        shutil.copy(os.path.join(source_class_dir, file), os.path.join(val_class_dir, file))

# Split data for each class
split_data(os.path.join(source_dir, 'Violence'), os.path.join(train_dir, 'Violence'), os.path.join(val_dir, 'Violence'))
split_data(os.path.join(source_dir, 'NonViolence'), os.path.join(train_dir, 'NonViolence'), os.path.join(val_dir, 'NonViolence'))

print("Data split into train and validation sets!")


In [4]:
# Load the InceptionV3 model without the top layer
base_model = InceptionV3(weights='imagenet', include_top=False)

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)  # Binary classification (violent/non-violent)

# Create the final model
model = Model(inputs=base_model.input, outputs=x)

# Freeze the layers of the base model to prevent retraining them
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
